# Урок 3: Файнтюнинг классификатора с PEFT

**Задача:** Адаптация DistilBERT под датасет эмоций (emotion).

In [ ]:
!pip install -q transformers datasets peft accelerate

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

# 1. Загрузка модели и данных
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

dataset = load_dataset("dair-ai/emotion")
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

In [ ]:
# 2. LoRA конфигурация
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# 3. Токенизация и обучение
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tokenized_ds = dataset.map(tokenize, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_steps=50,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
)

trainer.train()